# Caso de Estudio en negocios:  Audiolibros

### Problema

Nos han dado datos obtenidos con una App para Audiolibros.  Lógicamente, tiene que ver únicamente con las versiones audio de libros.  Cada cliente en la base de datos ha hecho una compra por lo menos una vez, por eso está en la base de datos.  Deseamos crear un algoritmo de ML que nos permita predecir si un cliente volverá a hacer una compra a la empresa de Audiolibros.

La idea principal es que si un cliente tiene una baja probabilidad de regresar, no hay razón alguna para gastar dinero en hacerles anunicios a él/ella.  Si podemos enfocar nuestros esfuerzos SOLAMENTE en aquellos clientes que probablemente compren de nuevo, podemos hacer grandes ahorros.  Es más, este modelo puede identificar las métricas importantes para que un cliente regrese.  La identificación de nuevos clientes crea valor y oportunidades de crecimiento.

Se tiene un .csv que resume los datos.  Hay varias variables:

* Customer ID, 
* Book length overall (suma de la duración en minutos de todas las compras), 
* Book length avg (duración promedio, en minutos, de todas las compras), 
* Price paid_overall (suma del precio de todas las compras),
* Price Paid avg (precio promedo de todas las compras), 
* Review (una variable Booleana indicando si el cliente dejó una evaluación), 
* Review out of 10 (si el/la cliente dejó una evaluación, su calificación de 0 a 10), 
* Total minutes listened (total de minutos escuchados), 
* Completion (completación de 0 a 1), 
* Support requests (número de requerimientos de soporte técnico), y 
* Last visited minus purchase date (Ultima visita menos la fecha de compra, en días)

Estas son las entradas (excluyendo Customer ID, ya que este es completamente arbitrario.  Es más como un nombre que un número).

Las metas son una variable Booleana (0 ó 1).  Tomaremos un período de 2 años para nuestras entradas, y los siguientes 6 meses como metas.  De esta forma, en realidad estamos prediciendo si:  basado en los últimos 2 años de actividad e involucramiento, un cliente volverá a comprar dentro de los siguientes 6 meses.  Seis meses suena con un tiempo razonable.  Si no compran luego de 6 meses, lo más probable es que se han ido a la competencia o no les gustó el formato de Audiolibros para recibir información.

La tarea es simple:  crear un algoritmo de ML, que pueda predecir si un cliente volverá a comprar.

Este es un problema de clasificación con dos clases:  "no comprará" o "comprará", representado por 0s y 1s. 

## Pre-procesar los datos, balancear el conjunto de datos.  Crear los tres subconjuntos:  entrenamiento, validación, y prueba.  Guarda los subconjuntos creados en un formato "amigable" de tensores (e.g. nombre.npz) 

Como estos datos son de la vida real, habrá necesidad de pre-procesarlos algo.  Este es el código relevante, no es tan difícil, pero es crucial para la creación de un buen modelo.

Este código puede ser re-utilizado para la mayoría de conjuntos de datos organizados de la forma:  muchas entradas, y una celda conteniendo las metas (conjuntos de datos de aprendizaje supergrandes).  Hay que tomar en cuenta que un problema específico podría requerir pre-procesamiennto adicional.

Nótese que se podría eliminar la fila de encabezados.  Lo que nos interesa son los datos.

### Importar las librerías relevantes

Estaremos utilizando la librería **preprocessng de sklearn** ya que con esta será más fácil estandarizar los datos

In [ ]:
import numpy as np
from sklearn import preprocessing

### Extraer los datos del csv

Las entradas son todas las columnas en el csv, exceptuando la primera [:, 0].  Esta es solo la ID del cliente, que ya dijimos es arbitraria y no aporta información útil.  

La última columna tampoco es de las entradas ya esta contiene las metas [:, -1].  Esta es la forma convencional de organizar los datos.

In [ ]:
datos_burdos = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

entradas_no_estandarizadas = datos_burdos[:, 1:-1]

metas = datos_burdos[:, -1]

### Barajear los datos

Cuando este conjunto de datos fue recolectado, se ordenó por fecha.

Es buena práctica "barajear" los indices antes de hacer el balanceo (ver nota, en sección de balanceo).  Esto es para eliminar cualquier efecto que pudiera ocurrir por tener los datos ordenados de alguna forma (por ejemplo, efecto del día...lunes es más lento en ventas, etc.).

Luego se usan los índices barajeados para reordenar los datos-

La práctica de barajear es más importante aún, cuando se implementa el método de tandas, ya que queremos que cada tanda sea lo más uniformemente distribuída al azar, que sea posible.

Sin embargo, todavía será necesario barajearlos después de balancear el conjunto de datos ya que todas las metas que contenga 1s estarán en las metas de entrenamiento.

Este código no es el óptimo, pero es el más simple de implementar.  Sin embargo, como el pre-procesamiento se hace una sola vez, velocidad no es algo a lo que estamos apuntando.



In [ ]:
indices_barajeados = np.arange(entradas_no_estandarizadas.shape[0])
np.random.shuffle(indices_barajeados)

entradas_no_estandarizadas = entradas_no_estandarizadas[indices_barajeados]
metas = metas[indices_barajeados]

### Balancear el conjunto de datos

La probabilidad inicial de asignar una clase (meta) a una observación es conocida como "previo" (**prior** en inglés).  Por ejemplo, si se está desarrollando un algoritmo ML para clasificar imágenes de perros y gatos, y los datos de entrenamientos tienen el 90% de imágenes de perros y el 10% de imágenes de gatos, se dice que los "priors", son de 0.9 para perros y 0.1 para gatos.

Con algoritmos de ML, es importante que el conjunto de datos esté balanceado en cuanto a las clases de las metas.  Si no es así, el algoritmo se torna "haragan" y simplemente favorece a la clase que aparezca con mayor frecuencia.  Lo ideal es que si son dos clases, las metas estén balanceadas 50% unos y 50% ceros.  Muchas veces no se puede lograr exactamente esta division pero se considera buena práctica que el balance esté entre el 45 y el 55%.

1. contar cuantos 1s hay en las metas
2. inicializar un contador para los 0s en las metas
3. inicializar una lista para contener los índices de las observaciones a eliminar
4. contar el número de metas que son 0s, una vez hay tantos como 1s hay, marcar esas observaciones
5. crear dos variables nuevas, una que contendrá las entradas y otra para las metas.
6. borrar todos los índices marcados para eliminar

In [ ]:
num_metas_1 = int(np.sum(metas))

contador_metas_0 = 0

indices_a_remover = []

for i in range(metas.shape[0]):
    if metas[i] == 0:
        contador_metas_0 += 1
        if contador_metas_0 > num_metas_1:
            indices_a_remover.append(i)

entradas_no_estandarizadas_prior_igual = np.delete(entradas_no_estandarizadas, indices_a_remover, axis = 0)
metas_prior_igual = np.delete(metas, indices_a_remover, axis = 0)

### Estandarizar las entradas

Este es el único lugar donde utilizaremos la funcionalidad de **sklearn**.  Sacaremos provecho de sus capacidades de pre-procesamiento.  Al final del ejercicio se podría intentar correrlo de nuevo SIN esta línea de código.  El resultado será interesante.

In [ ]:
entradas_estandarizadas = preprocessing.scale(entradas_no_estandarizadas_prior_igual)

### Barajear los datos

Cuando se recolectaron los datos, estos fueron ordenados por fecha.  Se barajean los índices de los datos para que no estén ordenados de forma alguna cuando se alimenten al modelo.  Como se usarán **tandas**, se desea que los datos estén distribuidos tan al azar como sea posible.

Luego se usarán los indices barajeados para barajear las entradas y metas mismas

In [ ]:
indices_barajeados = np.arange(entradas_estandarizadas.shape[0])
np.random.shuffle(indices_barajeados)

entradas_barajeadas = entradas_estandarizadas[indices_barajeados]
metas_barajeadas = metas_prior_igual[indices_barajeados]

### Dividir el conjunto de datos en subconjuntos de entrenamiento, validación, y prueba

Se asume que se va a querer una división de 80-10-10

Se balanceó el conjunto de datos para que las metas fuerab 50-50 (0s y 1s), pero los subconjuntos de entrenamiento, validacion, y prueba fueron tomados de un conjunto barajeado.  Se debe revisar que estos también estén balanceados.

Notar que cada vez que se ejecute este código se tendrán valores diferentes ya que se barajean en forma al azar.  Normalmente se pre-procesa una sola vez por lo que no será necesario volver a hacerlo.  Si se ejecuta todo el Notebook de nuevo, los .npz serán re-grabados con los datos nuevos

Se imprimen el número de metas que son 1s, el número total de observaciones, y la proporcióm para entrenamiento, validación, y prueba.

In [ ]:
tot_obs = entradas_barajeadas.shape[0]

obs_entreno = int(0.8 * tot_obs)
obs_validacion = int(0.1 * tot_obs)

obs_prueba = tot_obs - obs_entreno - obs_validacion

entradas_entreno = entradas_barajeadas[:obs_entreno]
metas_entreno = metas_barajeadas[:obs_entreno]

entradas_validacion = entradas_barajeadas[obs_entreno: obs_entreno + obs_validacion]
metas_validacion = metas_barajeadas[obs_entreno: obs_entreno + obs_validacion]

entradas_prueba = entradas_barajeadas[obs_entreno + obs_validacion:]
metas_prueba = metas_barajeadas[obs_entreno + obs_validacion:]

print(np.sum(metas_entreno), obs_entreno, np.sum(metas_entreno) / obs_entreno)
print(np.sum(metas_validacion), obs_validacion, np.sum(metas_validacion) / obs_validacion)
print(np.sum(metas_prueba), obs_prueba, np.sum(metas_prueba) / obs_prueba)

### Guardar los tres subconjuntos en *.npz

En el futuro se verá que es muy valioso nombrarlos de una forma coherente

In [ ]:
np.savez('Audiolibros_datos_entreno', entradas = entradas_entreno, metas = metas_entreno)
np.savez('Audiolibros_datos_validacion', entradas = entradas_validacion, metas = metas_validacion)
np.savez('Audiolibros_datos_prueba', entradas = entradas_prueba, metas = metas_prueba)